In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import plotly.graph_objects as go
import pandas as pd

pd.options.plotting.backend = 'plotly'

In [3]:
from scopone_scientifico import Tournament

In [5]:
tournament = Tournament(
    team1=('A', 'B'), 
    team2=('C', 'D'),
    n_match=100
)

tournament.run()